In [43]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [110]:
# url is to the RAW csv file on my github for convenience
url = 'https://raw.githubusercontent.com/nadiaenh/Dysarthria-Classifier/main/PD_remove_Nadia_update.csv?token=ASKTKFRHIMG6N54VCBV6ZMLAX3M5G'
df = pd.read_csv(url)
df.head()

,Status,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,RPDE,DFA,PPE
0,1,0,0.064241,0.074399,0.044886,0.069475,0.085062,0.097701,0.079287,0.056436,0.058637,0.079267,0.328638,0.097793,0.155900
1,1,0,0.027962,0.036188,0.020860,0.022598,0.064691,0.073276,0.051720,0.044440,0.059611,0.051699,0.348330,0.144300,0.097233
2,1,0,0.046101,0.053720,0.029204,0.035404,0.051549,0.074234,0.035577,0.032900,0.051276,0.035556,0.381812,0.085362,0.212444
3,1,0,0.050812,0.058215,0.037261,0.032912,0.075423,0.144157,0.058674,0.054209,0.069498,0.058632,0.412583,0.181761,0.350903
4,1,0,0.031469,0.043605,0.017983,0.015819,0.052604,0.071839,0.032162,0.037202,0.064302,0.032121,0.393664,0.134202,0.193781


In [111]:
# fitting the LogReg model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = df.iloc[:,1:]
y = df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y)
logReg = LogisticRegression(solver='lbfgs')
logReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [112]:
# predicting the test set 
y_pred = logReg.predict(X_test)

In [114]:
# performance metrics : confusion matrix 
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, y_pred) # [TN, FN], [TP, FP]
matrix
# added metrics on this are coming soon :
# results = (matrix[0][0] + matrix[1][0]) / df.row 
# print("Accuracy: %.3f" % (results))

array([[   7,   45],
       [   3, 1603]])

In [115]:
# performance metrics : kfolds cross validation (closer to 100 is better)
from sklearn import model_selection
for k in range(2,30,4):
  kfold = model_selection.KFold(n_splits=k, shuffle=True)
  results = model_selection.cross_val_score(logReg, X, y, cv=kfold, scoring='accuracy')
  print("Accuracy: %.3f (%.f)" % (results.mean(), k))

Accuracy: 0.972 (2)
Accuracy: 0.973 (6)
Accuracy: 0.973 (10)
Accuracy: 0.973 (14)
Accuracy: 0.973 (18)
Accuracy: 0.974 (22)
Accuracy: 0.973 (26)


In [106]:
# performance metrics : log loss (closer to 0 is better)
for k in range(2,30,4):
  kfold = model_selection.KFold(n_splits=k, shuffle=True)
  results = model_selection.cross_val_score(logReg, X, y, cv=kfold, scoring='neg_log_loss')
  print("Logloss: %.3f (%.f)" % (results.mean(), k))

Logloss: -0.070 (2)
Logloss: -0.068 (6)
Logloss: -0.068 (10)
Logloss: -0.068 (14)
Logloss: -0.068 (18)
Logloss: -0.067 (22)
Logloss: -0.067 (26)


In [107]:
# performance metrics : area under curve (1 is perfect predictions, 0.5 is as good as random)
for k in range(2,30,4):
  kfold = model_selection.KFold(n_splits=k, shuffle=True)
  results = model_selection.cross_val_score(logReg, X, y, cv=kfold, scoring='roc_auc')
  print("AUC: %.3f (%.f)" % (results.mean(), k))

AUC: 0.953 (2)
AUC: 0.955 (6)
AUC: 0.952 (10)
AUC: 0.948 (14)
AUC: 0.953 (18)
AUC: 0.955 (22)
AUC: 0.952 (26)


In [108]:
# performance metrics : classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# precision = for all instances classified positive, what percent was correct?
# recall = for all instances that were actually positive, what percent was classified correctly?
# f1 = weighted harmonic mean of precision and recall (1 is best, 0 is worst)
# support = number of actual occurrences of the class in the dataset

              precision    recall  f1-score   support

           0       0.60      0.15      0.24        41
           1       0.98      1.00      0.99      1617

    accuracy                           0.98      1658
   macro avg       0.79      0.57      0.61      1658
weighted avg       0.97      0.98      0.97      1658

